In [ ]:
# %%shell
# set -e

# LG=$1
# WIKI_DUMP_NAME=${LG}wiki-latest-pages-articles.xml.bz2
# WIKI_DUMP_DOWNLOAD_URL=https://dumps.wikimedia.org/${LG}wiki/latest/$WIKI_DUMP_NAME

# # download latest Wikipedia dump in chosen language
# echo "Downloading the latest $LG-language Wikipedia dump from $WIKI_DUMP_DOWNLOAD_URL..."
# wget -c $WIKI_DUMP_DOWNLOAD_URL
# echo "Succesfully downloaded the latest $LG-language Wikipedia dump to $WIKI_DUMP_NAME"

# Setup

In [ ]:
!pip install wikipedia
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=42a886f535a5114217defd838113ee2d9a8c75626316456cf49c1bff78de2c2e
  Stored in directory: /root/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 476 kB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-1vrlk74v/farm-haystack_5fd9b4a05d9d4e359be6334a68ce7f40
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-

In [ ]:
import logging
from haystack.utils import launch_es

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

launch_es()

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
# ElasticSearch will try to open a connection in the constructor so we have to wait 30secs just to make sure it is ready
import time
time.sleep(30)

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


# Get Data

In [ ]:
import wikipedia
import re
import os

doc_dir='/data'
doc_name='wiki_data.txt'
file_path = os.path.join(doc_dir, doc_name)

text = wikipedia.page('breakingbad').content
# text = re.sub(r'==.*?==+', '', text)
# text = text.replace('\n', '')
# text = text.replace('\\', '')

# Create director if it doesn't exist
if not os.path.isdir('data'):
  os.mkdir('data')

# Write data to txt
with open('data/wiki_data.txt', "w") as f:
    f.write(text)
    f.close()

# Process Data

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs

# doc_dir = 'data/wikipedia'

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path='data', clean_func=clean_wiki_text, split_paragraphs=True)

document_store.write_documents(docs)

INFO:haystack.utils.preprocessing:Converting data/wiki_data.txt


In [ ]:
docs[:2]

[<Document: {'content': "Breaking Bad is  an American crime drama television series created and produced by Vince Gilligan. Set and filmed in Albuquerque, New Mexico, the series follows Walter White (Bryan Cranston), an underpaid, overqualified, and dispirited high-school chemistry teacher who is struggling with a recent diagnosis of stage-three lung cancer. White turns to a life of crime and partners with a former student, Jesse Pinkman (Aaron Paul), to produce and distribute methamphetamine to secure his family's financial future before he dies, while navigating the dangers of the criminal underworld. The show aired on AMC from January 20, 2008, to September 29, 2013, consisting of five seasons for a total of 62 episodes.\nAmong the show's co-stars are Anna Gunn and RJ Mitte as Walter's wife Skyler and son Walter Jr., and Betsy Brandt and Dean Norris as Skyler's sister Marie Schrader and her husband Hank, a DEA agent. Others include Bob Odenkirk as Walter's and Jesse's lawyer Saul Go

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
from haystack.nodes import FARMReader

# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(
    query="How many seasons are in the show", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [ ]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'three', 'type': 'extractive', 'score': 0.8200069665908813, 'context': 'ian Johnson\'s experience on the show ===\nDirector Rian Johnson worked on three episodes ("Fly", "Fifty-One" and "Ozymandias") and in an interview with', 'offsets_in_document': [{'start': 79, 'end': 84}], 'offsets_in_context': [{'start': 73, 'end': 78}], 'document_id': '459a43ea958b6ce83c0a70bcdb6e66a3', 'meta': {'name': 'wiki_data.txt'}}>,
             <Answer {'answer': 'fifth', 'type': 'extractive', 'score': 0.757225751876831, 'context': ' been warned that the police are onto him. During the second part of the fifth season, white supremacist Jack Welker says "don\'t skimp on family", and', 'offsets_in_document': [{'start': 2444, 'end': 2449}], 'offsets_in_context': [{'start': 73, 'end': 78}], 'document_id': 'b20e12742c67333c26492632ec1ddc53', 'meta': {'name': 'wiki_data.txt'}}>,
             <Answer {'answer': 'five', 'type': 'extractive', 'score': 0.7274923920631409, 'context': 